modified from https://huggingface.co/agents-course/notebooks/blob/main/unit2/smolagents/code_agents.ipynb

In [3]:
USERNAME = 'CoralLeiCN'

In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool
from utils import gemini_model
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=gemini_model)

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="music recommendations party Wayne's mansion")                                 
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Similar Songs Finder | Spotify Playlist Generator - Chosic](https://www.chosic.com/playlist-generator/)
Need to find similar songs to a song you like? To music you love? A playlist created for you in seconds! With 
Spotify Playlist Generator online tool, create awesome playlists in seconds, featuring similar songs to those you 
love. Whether by songs, artists, genres, moods, or based on a playlist, just start with what you have in mind, and 
it will provide you with plenty of song recommendations ...

[Party on Wayne - playlist by Music League | Spotify](https://open.spotify.com/playlist/66wXTng3HRUePKf9WdOrrK)
Party on Wayne · Playlist · 20 songs · 6 likes

[Music Taste Analyzer - Chosic](https://www.chosic.com/music-taste-checker/)
Find out your favorite genres and moods, and get music recommendations that will help you discover new music. 
Understand your music personality and uncover interesting facts about your taste in music. Simply enter the artists
you love, and we will generate a chart of your top genres and moods. We will also provide a few song 
recommendations ...

[Party on, Wayne! - playlist by Music League | Spotify](https://open.spotify.com/playlist/4J61XoHr2CINqRA1DV0ga7)
Party on, Wayne! · Playlist · 17 songs · 4 likes

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
A party planner's list PA system/speakers. An epic party needs an epic PA system! When selecting a sound system for
your party, the size of the room is crucial. A small room might need just a pair of compact speakers, while a 
larger space could benefit greatly from a subwoofer to boost bass frequencies.

[Party on, Wayne! - YouTube Music](https://music.youtube.com/playlist?list=PLXxuECYNOaw87L8fiV0UVlHQupKEwpIha)
A new music service with official albums, singles, videos, remixes, live performances and more for Android, iOS and
desktop. It's all here.

[Various - Music From The Motion Picture Wayne's 
World](https://www.discogs.com/release/8033702-Various-Music-From-The-Motion-Picture-Waynes-World)
View credits, reviews, tracks and shop for the 2016 Vinyl release of "Music From The Motion Picture Wayne's World" 
on Discogs. Everything Releases Artists Labels. Advanced Search ... Black Sabbath - Time Machine (Wayne's World) 
4:19; Lists Add to List. ... Party on!! Reply Helpful. eblanken Jun 3, 2016. Report; Update on the UO release: I 
...

[50+ Best Songs For Your Next Party in 2025 - Aleka's 
Get-Together](https://alekasgettogether.com/top-songs-for-any-party/)
Whether you're planning a How To Host The Best Party At Home or just want to liven up your playlist, we've got you 
covered with 100 crowd-pleasing tunes. From timeless classics to modern hits, these songs are guaranteed to keep 
the energy high and the good times rolling all night long. 50 Popular Party Songs. Uptown Funk - Mark Ronson ft ...

[you're studying at night in bruce wayne's mansion - YouTube](https://www.youtube.com/watch?v=i4vjSoUH8bE)
you're studying at night in bruce wayne's mansion... hopefully alfred's making you coffee soonlike and subscribe 
for weekly dark academia study music uploads...

[Live set Wayne Brady Mansion, by Damion Shade 
(Demos)](https://damionshadeandtheboombapchorus.bandcamp.com/album/live-set-wayne-brady-mansion)
Live set Wayne Brady Mansion by Damion Shade (Demos), released 24 February 2020 1. Mic Check Boom Bap 2. Heathens 
Boom Bap newest 3. Float Boom Bap 4. Rocky Balboa (New Demo) 5. So Gone Boom Bap (Louder) 6. Ordinary Boom Bap 
(Live Demo) 7. A Hoodie & Skittles Boom Bap Newest 8. Heathens (Plus Guitar) 9. Float (Master) 10. A Hoodie &; 
Skittles (New Demo) These are just simple phone demos, but ...

Out: None

[Step 1: Duration 5.37 seconds| Input tokens: 2,249 | Output tokens: 77]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Here are some music recommendations for a party at Wayne's mansion: Classical: Gymnopédie 1 by     
  Erik Satie; Smooth Jazz: Music by Sade, instrumental jazz standards; Easy Listening: My Baby Just Cares for Me,  
  Can't Help Falling in Love by Elvis Presley; General Sophisticated Dinner Party Mix: Include artists like Al     
  Green and selections from the Dinner Party Sophisticated Mix playlist on Spotify.")                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Here are some music recommendations for a party at Wayne's mansion: Classical: Gymnopédie 1 by 
Erik Satie; Smooth Jazz: Music by Sade, instrumental jazz standards; Easy Listening: My Baby Just Cares for Me, 
Can't Help Falling in Love by Elvis Presley; General Sophisticated Dinner Party Mix: Include artists like Al Green 
and selections from the Dinner Party Sophisticated Mix playlist on Spotify.

[Step 3: Duration 2.12 seconds| Input tokens: 10,445 | Output tokens: 524]

"Here are some music recommendations for a party at Wayne's mansion: Classical: Gymnopédie 1 by Erik Satie; Smooth Jazz: Music by Sade, instrumental jazz standards; Easy Listening: My Baby Just Cares for Me, Can't Help Falling in Love by Elvis Presley; General Sophisticated Dinner Party Mix: Include artists like Al Green and selections from the Dinner Party Sophisticated Mix playlist on Spotify."

Using a Custom Tool to Prepare the Menu

In [ ]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

agent = CodeAgent(tools=[suggest_menu], model=HfApiModel())

agent.run("Prepare a formal menu for the party.")

Using Python Imports Inside the Agent


In [ ]:
from smolagents import CodeAgent, HfApiModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=HfApiModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

Sharing Our Custom Party Preparator Agent to the Hub

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

In [ ]:
# agent.push_to_hub(f'{USERNAME}/AlfredAgent')

In [ ]:
# agent = CodeAgent(tools=[], model=HfApiModel())
# alfred_agent = agent.from_hub(f"{USERNAME}/AlfredAgent", trust_remote_code=True)
# alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")


Inspecting Our Party Preparator Agent with OpenTelemetry and Langfuse 📡

In [ ]:
import os
import base64
from google.colab import userdata

LANGFUSE_PUBLIC_KEY=os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY=os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(tools=[], model=HfApiModel())
alfred_agent = agent.from_hub('sergiopaniego/AlfredAgent', trust_remote_code=True)
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")